# Objective: Rolling 1 week window of all active client NA data. 
Pull date from curated_task_fields and merge that with the product data, to find the products that are receving NA data. 

In [1]:
import pandas as pd

#Get the previous date
from datetime import date, timedelta, datetime

#Interacting with Postgres SQL
import psycopg2
from tqdm import tqdm

import time

#importing database
from database_config import postgres as cfg

# Establishing Connection to Postgres via psycopg2

In [2]:
#Setting up the connection to the PostgreSQL -feeddate
conn = psycopg2.connect(**cfg)

#this works with the last days of the month as well. 
todays_date = str(datetime.today())[:10]

In [3]:
active_client_query = ("""
SELECT 
    id,
    name
FROM "public"."customers" 
WHERE active = 't';
""")

active_client = pd.read_sql(active_client_query,conn)

In [4]:
rel_client = next(iter(active_client.to_dict().values()))

rel_query = ("""SELECT 
    DISTINCT ct.customer_id
FROM "curation_tasks" AS ct
WHERE started_at >= CURRENT_TIMESTAMP - INTERVAL '1 week'
AND (ct.resolution IS NULL OR ct.resolution = 'misclassified') 
LIMIT 100""")

rel_client = pd.read_sql(rel_query,conn)

rel_client_weekly = list(rel_client['customer_id'])

active_client = active_client[active_client['id'].isin(rel_client_weekly)]

active_client.set_index('id',drop=True,inplace=True)

important_customers = next(iter(active_client.to_dict().values()))

In [5]:
important_customers

{77: 'containerstore',
 26: 'discountschoolsupply',
 83: 'swansonhealth',
 60: 'watchdog',
 39: 'josabank',
 21: 'menswearhouse',
 13: 'pier1',
 32: 'eddiebauer',
 16: 'cb2',
 71: 'burlington',
 64: 'tractorsupply',
 11: 'crateandbarrel',
 55: 'cvs'}

In [20]:
def NA_report(customer_id,timeframe):
    """
    Parameters:
    ------------------------------------------------
    customer_id: ensure it matches the SQL database
    
    timeframe: ensure this is a string. examples('1 month','5 weeks','1 year')
    
    Details:
    """

    company_na_reports = (  
    '''SELECT 
        ct."id" AS curation_task_id, 
        ct.customer_id, 
        ct.resolution, 
        --- truncated the timestamp from the date for aggregation.
        date_trunc('day',ct.started_at) as "started_at",
        cpf.product_id,
        cpf."name" AS attribute_name, 
        cpf."Total_NAs", 
        cpf.curation_tasks_count
    FROM "curation_tasks" AS ct

    INNER JOIN 
        (SELECT 
            cpf.curation_task_id,
            cpf.customer_id,
            cpf.product_id,
            cpf.name, 
            SUM(CASE WHEN cpf.value = 'n/a' THEN 1 ELSE 0 END) as "Total_NAs",
            COUNT(cpf.id) AS curation_tasks_count
        FROM "public"."curated_product_fields"  as cpf
        WHERE customer_id = {}
        GROUP BY
            cpf.curation_task_id,    
            cpf.customer_id,
            cpf.product_id,
            cpf.name
        ) AS cpf
    ON ct.id = cpf.curation_task_id

    WHERE started_at >= CURRENT_TIMESTAMP - INTERVAL {!r}
    --- specific resolution that are not taken care of my rules or bulk
    AND (ct.resolution IS NULL OR ct.resolution = 'misclassified') 
    AND ct.customer_id = {}
    ORDER BY started_at;
    ''')

    #creating blank data set to append to. 
    master_data = pd.DataFrame()

    print("process has begun")
    for cust in customer_id:
        #Returning the data in pandas to export it as a CSV.
        data = pd.read_sql(company_na_reports.format(cust,timeframe,cust),conn)

        #if the there is data, convert the date to a date element and append the customer name
        if data.shape[0] != 0:
            data['started_at'] = data['started_at'].dt.date
            data['customer_name']= important_customers[cust]
            print(cust, 'query and data clean completed')
        else:
            print('No data available in your time frame for Cust_id ',cust)
            pass
        master_data = master_data.append(data,sort=False)
        time.sleep(1)
        
    if master_data.shape[0] != 0:
        master_data.to_csv('C:/Users/groupby/Documents/Github/NA_Report/data/customer_'+str(customer_id)+'_'+timeframe+'_NA_report.csv',index=False)
    return master_data

In [21]:
weekly_report = pd.read_csv('./data/Weekly_NA_report.csv')

In [22]:
foo= 'customer_id	bucket_id	bucket_name	product_id	product_name	active	external_id	image_url	strategy_bucket_attribute_id	strategy_bucket_id	attribute_id	family_friendly'


In [23]:
foo.replace('	',',')

'customer_id,bucket_id,bucket_name,product_id,product_name,active,external_id,image_url,strategy_bucket_attribute_id,strategy_bucket_id,attribute_id,family_friendly'

In [24]:
unique_prod_list = list(master_data['product_id'].unique())
uni_prod_sql = str(unique_prod_list).replace("[",'').replace("]",'')

NameError: name 'master_data' is not defined

In [25]:
relevant_customers = list(important_customers.keys())

# Querying N/A Product data

In [6]:
query_product= ('''SELECT
    p.customer_id,
    pbb.bucket_name,
    p.product_id,
    p.product_name,
    p.active,
    p.external_id,
    p.image_url,
    sba.bucket_id,
    sba.attribute_id,
    sba.family_friendly
FROM (SELECT
            p.id as product_id,
            p.active,
            p.customer_id,
            p.name as product_name,
            p.external_id,
            p.image_url
        FROM products as p
        WHERE p.customer_id  = {}
        AND p.active = 't'
        AND p.id in ({})) AS p
INNER JOIN (SELECT
            pb.product_id
            ,pb.bucket_id
            ,buc.bucket_name
            FROM products_buckets AS pb
            INNER JOIN (SELECT
                            id AS bucket_id
                            ,name AS bucket_name
                        FROM buckets) AS buc
            ON buc.bucket_id = pb.bucket_id) AS pbb
ON p.product_id = pbb.product_id
INNER JOIN (SELECT 
                sb.bucket_id, 
                sxa.attribute_id, 
                family_friendly 
                FROM strategy_buckets as sb
                    INNER JOIN (SELECT 
                                    id, 
                                    strategy_bucket_id, 
                                    attribute_id,
                                    family_friendly 
                                    FROM strategy_buckets_attributes) as sxa
                    ON sb.id = sxa.strategy_bucket_id) as sba
ON pbb.bucket_id = sba.bucket_id
''')

In [7]:
#reseting the data
master_data_prod = pd.DataFrame(columns=['customer_id','bucket_id','bucket_name','product_id','product_name','active','external_id','image_url','attribute_id','family_friendly'])
master_data_prod.to_csv('C:/Users/groupby/Documents/Github/NA_Report/data/Weekly_products.csv',mode='w',index=False,header=True)
counter = 0 

In [8]:
for cust in relevant_customers:
    #Returning the data in pandas to export it as a CSV.
    data_prod = pd.read_sql(query_product.format(cust,uni_prod_sql),conn,chunksize=1000)
    
    print('Cust_id ',cust, 'Product query and data clean completed')
    
    #master_data_prod = master_data_prod.append(data_prod,sort=False)
    data_prod.to_csv('C:/Users/groupby/Documents/Github/NA_Report/data/Weekly_products.csv',mode='a',index=False,header=False)
    counter += 1
    print(str(counter)+' completed of '+str(len(relevant_customers)))
    #time.sleep(3)

NameError: name 'relevant_customers' is not defined